# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Predictions</span>


## 🗒️ In this notebook we will see how to create a training dataset from the feature groups: 

1. Loading the training data.
2. Train the model.
3. Register model in Hopsworks model registry.

![part3](images/03_model.png) 

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [84]:
import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/167
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🪝 Feature View and Training Dataset Retrieval </span>

In [2]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 1
)

In [3]:
train_data = feature_view.get_training_data(1)[0]

train_data.head()

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,2,16,1663027200000,0.296339,-0.769711,-0.228696,0.174822,-0.262875,-0.295896,-0.095551,...,1.292511,0.49434,-0.629129,-0.94677,1.251215,-1.073768,-1.719523,-1.726476,-1.878691,2
1,1,4,1662584400000,0.809041,1.236292,-0.41581,-2.566292,-0.047795,0.430394,0.217162,...,-0.875157,-0.986137,-0.688734,1.13537,-1.721077,0.602495,0.423267,0.419426,0.285561,0
2,1,19,1662940800000,-0.899964,0.344735,2.390909,0.723045,0.167284,-0.295896,0.68623,...,-0.610403,-0.512995,-0.757281,0.284408,-0.321398,-1.937297,-0.079925,-0.04877,0.285561,0
3,2,8,1662584400000,-0.764249,1.28087,-0.41581,-0.949035,-0.262875,-0.295896,0.373518,...,-0.941345,-1.154026,-0.092678,1.244003,0.402761,0.602495,0.134522,0.107295,0.285561,0
4,1,16,1662930000000,-1.101023,0.344735,1.829565,0.997157,0.167284,-0.295896,0.68623,...,-0.701412,-0.757197,-0.761751,0.293461,0.073106,-1.327747,0.011257,0.029262,0.285561,0


_____
## PyTorch LSTM

---

## <span style="color:#ff5f27;"> 🧬 SequenceDataset </span>

In [85]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [30]:
class SequenceDataset(Dataset):
    def __init__(self, data, target='aqi', sequence_length=5):
        self.features = [column for column in [*data.columns] if column != target]
        self.target = target
        self.sequence_length = sequence_length
        self.X = torch.tensor(data.drop(target, axis = 1).values.astype(np.float32)) 
        self.y = torch.tensor(data[self.target])
        
    def __repr__(self):
        return f'Features: {self.features}\nTarget: {self.target}'
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self,i):
        if i < self.sequence_length:
            dist = self.sequence_length - i+1
            X = [*[self.X[0] for i in range(dist-2)],*self.X[:i+1]]
            return torch.tensor([[*obs] for obs in X]).float(),self.y[i].float()
        return self.X[i+1 - self.sequence_length:i+1].float(),self.y[i].float()

In [88]:
test_data = train_data.sort_values(by=["date", "city"], ascending=[False, True]).head(4)

In [94]:
test_data.shape

(4, 42)

In [120]:
df_torch_test = SequenceDataset(
    data = test_data,
    target = 'aqi',
    sequence_length = 1
)

print(df_torch_test.X[1])
print(df_torch_test.y[1])

print(df_torch_test.X[0])
print(df_torch_test.y[0])

tensor([ 1.0000e+00,  1.6632e+12,  5.0745e-01, -1.9287e+00, -4.1581e-01,
        -1.1957e+00,  1.2427e+00,  6.7249e-01,  9.9894e-01, -3.5798e-01,
        -2.9577e-01, -3.1088e-01, -4.0398e-01, -5.2808e-01, -3.0197e-01,
        -3.5355e-01, -3.5355e-01, -3.5355e-01, -9.0578e-01, -7.8329e-02,
        -6.1143e-01, -9.0578e-01, -3.7831e-01, -6.9183e-01, -1.6421e-01,
         2.1185e-01, -5.2007e-01, -3.1363e-01, -6.4840e-01,  0.0000e+00,
         0.0000e+00,  4.3206e-02, -1.0090e-01,  1.0175e+00, -2.0874e+00,
        -2.9978e-01,  6.0250e-01, -9.3096e-01, -9.4615e-01, -7.9656e-01,
         0.0000e+00])
tensor(4)
tensor([0.0000e+00, 1.6632e+12, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
        1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
        1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
        1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
        1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+

In [6]:
df_torch = SequenceDataset(
    data = train_data,
    target = 'aqi',
    sequence_length = 5
)

print(df_torch.X[0])
print(df_torch.y[0])

tensor([ 2.0000e+00,  1.6630e+12,  2.9634e-01, -7.6971e-01, -2.2870e-01,
         1.7482e-01, -2.6288e-01, -2.9590e-01, -9.5551e-02, -1.1894e+00,
        -1.5127e+00, -1.3768e+00, -9.3865e-01, -1.1514e+00, -7.9609e-01,
        -3.5355e-01, -3.5355e-01, -3.5355e-01, -9.8886e-01,  5.5164e-01,
        -1.7699e-01, -9.8886e-01,  5.9449e-01, -8.3310e-02,  8.8902e-01,
         1.2962e+00,  3.0990e+00,  1.2638e+00,  2.6862e+00,  0.0000e+00,
         0.0000e+00,  1.2925e+00,  4.9434e-01, -6.2913e-01, -9.4677e-01,
         1.2512e+00, -1.0738e+00, -1.7195e+00, -1.7265e+00, -1.8787e+00,
         2.0000e+00])
tensor(16)


In [86]:
df_torch[2]

(tensor([[ 0.0000e+00,  1.6632e+12,  2.0586e-01, -1.5008e+00, -2.2870e-01,
          -5.9269e-01,  1.6728e+00,  1.3988e+00,  1.6244e+00,  8.8918e-01,
           3.1267e-01,  1.8209e+00, -4.0398e-01, -2.9434e-01,  2.7451e-02,
          -3.5355e-01, -3.5355e-01, -3.5355e-01, -3.3462e-02,  9.1161e-01,
           5.4707e-01, -3.3462e-02,  9.0168e-01,  5.6868e-01, -6.2288e-02,
          -6.5562e-01, -5.2007e-01,  1.0392e+00, -6.4840e-01,  0.0000e+00,
           0.0000e+00,  2.2771e+00,  1.9290e+00,  1.3423e+00, -1.2998e+00,
          -1.2617e+00,  6.0250e-01,  4.5028e-01,  4.3893e-01,  2.8556e-01,
           1.0000e+00],
         [ 0.0000e+00,  1.6632e+12,  2.0586e-01, -1.5008e+00, -2.2870e-01,
          -5.9269e-01,  1.6728e+00,  1.3988e+00,  1.6244e+00,  8.8918e-01,
           3.1267e-01,  1.8209e+00, -4.0398e-01, -2.9434e-01,  2.7451e-02,
          -3.5355e-01, -3.5355e-01, -3.5355e-01, -3.3462e-02,  9.1161e-01,
           5.4707e-01, -3.3462e-02,  9.0168e-01,  5.6868e-01, -6.2288e-02,
 

In [8]:
df_torch[5]

(tensor([[ 1.0000e+00,  1.6626e+12,  8.0904e-01,  1.2363e+00, -4.1581e-01,
          -2.5663e+00, -4.7795e-02,  4.3039e-01,  2.1716e-01, -3.5798e-01,
          -2.9577e-01, -8.4382e-01, -5.1091e-01, -4.5017e-01, -3.0197e-01,
           2.8284e+00,  2.8284e+00,  2.8284e+00, -2.8270e-01, -1.2183e+00,
          -9.4933e-01, -2.8270e-01, -1.1463e+00, -8.6570e-01, -9.4565e-01,
          -4.3875e-01, -5.2007e-01, -6.5182e-01, -6.4840e-01,  0.0000e+00,
           0.0000e+00, -8.7516e-01, -9.8614e-01, -6.8873e-01,  1.1354e+00,
          -1.7211e+00,  6.0250e-01,  4.2327e-01,  4.1943e-01,  2.8556e-01,
           0.0000e+00],
         [ 1.0000e+00,  1.6629e+12, -8.9996e-01,  3.4474e-01,  2.3909e+00,
           7.2305e-01,  1.6728e-01, -2.9590e-01,  6.8623e-01, -3.5798e-01,
          -6.0000e-01,  2.2206e-01, -5.1091e-01, -6.8391e-01, -3.0197e-01,
          -3.5355e-01, -3.5355e-01, -3.5355e-01,  1.0466e+00, -8.5828e-01,
          -1.2872e-01,  1.0466e+00, -6.0871e-01, -8.3310e-02,  3.9638e-02,
 

---

## <span style="color:#ff5f27;">🧑🏻‍🔬 DataLoader </span>

In [117]:
loader_test = DataLoader(df_torch_test,batch_size = 1)

In [118]:
loader_train = DataLoader(df_torch,batch_size = 3)

X, y = next(iter(loader_train))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([3, 5, 41])
Target shape: torch.Size([3])


---

## <span style="color:#ff5f27;">🤖 Model Building </span>

In [10]:
class LSTMModel(nn.Module):
    def __init__(self,n_features,num_layers,hidden_size):
        super().__init__()
        self.n_features=n_features
        self.num_layers=num_layers
        self.hidden_size=hidden_size
        
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden_size,
            batch_first=True,
            num_layers=self.num_layers
        )
        
        self.linear = nn.Linear(
            in_features=hidden_size,
            out_features=1
        )
        
    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  

        return out

In [11]:
model = LSTMModel(
    n_features=len(df_torch.X[0]),
    num_layers=32,
    hidden_size=16
)

learning_rate = 0.05
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

---

## <span style="color:#ff5f27;">👨🏻‍⚖️ Model Evaluation </span>

In [12]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = .0
    model.train()

    for X, y in data_loader:
        output = model(X)

        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):

    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}\n")

In [13]:
for epoch in range(10):
    print(f"Epoch {epoch}\n---------")
    train_model(loader_train, model, loss_function, optimizer=optimizer)

Epoch 0
---------
Train loss: 210.8397813240687
Epoch 1
---------
Train loss: 160.9134476284186
Epoch 2
---------
Train loss: 127.02985948324203
Epoch 3
---------
Train loss: 106.54613892237346
Epoch 4
---------
Train loss: 95.56713151931763
Epoch 5
---------
Train loss: 90.36365032196045
Epoch 6
---------
Train loss: 88.19353199005127
Epoch 7
---------
Train loss: 87.40594188372295
Epoch 8
---------
Train loss: 87.1639970143636
Epoch 9
---------
Train loss: 87.10609141985576


## <span style='color:#ff5f27'>👮🏼‍♀️ Model Registry</span>

In [115]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [15]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(train_data.drop('aqi',axis=1))
output_schema = Schema(train_data.aqi)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'city', 'type': 'object'},
   {'name': 'date', 'type': 'object'},
   {'name': 'iaqi_h', 'type': 'object'},
   {'name': 'iaqi_p', 'type': 'object'},
   {'name': 'iaqi_pm10', 'type': 'object'},
   {'name': 'iaqi_t', 'type': 'object'},
   {'name': 'o3_avg', 'type': 'object'},
   {'name': 'o3_max', 'type': 'object'},
   {'name': 'o3_min', 'type': 'object'},
   {'name': 'pm10_avg', 'type': 'object'},
   {'name': 'pm10_max', 'type': 'object'},
   {'name': 'pm10_min', 'type': 'object'},
   {'name': 'pm25_avg', 'type': 'object'},
   {'name': 'pm25_max', 'type': 'object'},
   {'name': 'pm25_min', 'type': 'object'},
   {'name': 'uvi_avg', 'type': 'object'},
   {'name': 'uvi_max', 'type': 'object'},
   {'name': 'uvi_min', 'type': 'object'},
   {'name': 'tempmax', 'type': 'object'},
   {'name': 'tempmin', 'type': 'object'},
   {'name': 'temp', 'type': 'object'},
   {'name': 'feelslikemax', 'type': 'object'},
   {'name': 'feelslikemin', 'type': 'object

In [16]:
model_dir = "./model"
torch.save(model, model_dir)

model_hops = mr.torch.create_model(
    name = "pytorch_model",
    model_schema = model_schema
)

model_hops.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/167/models/pytorch_model/1


Model(name: 'pytorch_model', version: 1)

## <span style="color:#ff5f27;">🚀 Fetch and test the model </span>

In [116]:
model

LSTMModel(
  (lstm): LSTM(41, 16, num_layers=32, batch_first=True)
  (linear): Linear(in_features=16, out_features=1, bias=True)
)

In [17]:
model = mr.get_model("pytorch_model", version = 1)
model_dir = model.download()

model = torch.load(model_dir + '/model')
model

LSTMModel(
  (lstm): LSTM(41, 16, num_layers=32, batch_first=True)
  (linear): Linear(in_features=16, out_features=1, bias=True)
)

In [18]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X).int()
            output = torch.cat((output, y_star), 0)

    return output

In [119]:
predict(loader_test, model)

tensor([12., 12., 12., 12.])

In [19]:
predict(loader_train,model)

tensor([12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
        12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
        12., 12., 12., 12., 12., 12., 12., 12.])

---

## sklearn GradientBossing 

In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [129]:
import warnings

# Mute warnings
warnings.filterwarnings("ignore")

In [130]:
train_data.columns

Index(['city', 'aqi', 'date', 'iaqi_h', 'iaqi_p', 'iaqi_pm10', 'iaqi_t',
       'o3_avg', 'o3_max', 'o3_min', 'pm10_avg', 'pm10_max', 'pm10_min',
       'pm25_avg', 'pm25_max', 'pm25_min', 'uvi_avg', 'uvi_max', 'uvi_min',
       'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
       'conditions'],
      dtype='object')

In [133]:
train_data = train_data.sort_values(by=["date", 'city'], ascending=[False, True]).reset_index(drop=True)

In [24]:
test = train_data.sort_values(by=["date", 'city'], ascending=[False, True]).head(20).reset_index(drop=True)
test.head()

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,0,23,1663200000000,0.205862,-1.500788,-0.228696,-0.59269,1.672842,1.39878,1.624368,...,2.277062,1.92903,1.342326,-1.299829,-1.261723,0.602495,0.450284,0.438934,0.285561,1
1,1,4,1663200000000,0.507451,-1.928735,-0.41581,-1.195735,1.242683,0.67249,0.998943,...,0.043206,-0.100903,1.017476,-2.087421,-0.299781,0.602495,-0.930962,-0.946148,-0.796565,0
2,2,6,1663200000000,1.11063,-2.374514,-0.602925,-1.30538,1.242683,1.156683,0.842587,...,0.580986,0.066986,-0.471174,-2.223213,0.997219,-0.972176,-1.22815,-1.219262,-1.337628,2
3,3,6,1663200000000,1.155868,-0.635977,-0.79004,-0.099289,0.167284,0.430394,-0.095551,...,-1.173004,-1.428754,0.200879,-0.729503,1.143131,1.262841,-1.731342,-1.745984,-1.878691,2
4,0,25,1663113600000,0.090253,-1.055009,-0.228696,0.010355,1.672842,1.156683,1.780725,...,1.739282,1.913767,1.322955,-1.028245,-0.002552,0.602495,-1.057605,-1.043689,-1.337628,1


In [141]:
train_data["aqi_next_day"] = train_data.groupby('city')['aqi'].shift(1)

In [153]:
train_data.head(5)

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,...,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions,aqi_next_day
0,0,7,1663718400000,0.454083,1.195838,-0.509993,-2.081702,-0.843871,-0.879904,-0.371219,...,-0.894416,-0.016729,1.352183,-2.705794,0.481224,1.070238,1.054494,0.985821,3,NaN
1,1,9,1663718400000,1.114657,1.287410,0.770920,-2.659991,-1.077559,-1.565213,-0.013258,...,-1.647902,0.751254,1.249014,-2.429295,0.481224,0.580380,0.553767,0.419498,0,NaN
2,2,6,1663718400000,0.013701,1.112590,-0.296508,-1.905701,-0.376496,-1.336777,0.165723,...,-1.075253,0.376752,1.137247,-0.746045,0.481224,0.062827,0.053040,-0.146824,0,NaN
3,0,19,1663632000000,0.178844,0.413309,-0.083022,-0.397120,-0.376496,-0.651467,-0.192239,...,-0.186139,1.490138,0.715974,-1.750174,0.481224,0.592497,0.573796,0.419498,0,7
4,1,4,1663632000000,0.399035,0.371686,-0.296508,-0.774265,-1.077559,-0.423031,-0.550200,...,0.416650,1.587559,0.509636,-0.202748,0.509843,-0.470305,-0.487745,-0.146824,1,9


In [154]:
train_data.shape

(54, 43)

In [156]:
X = train_data.dropna()

In [157]:
y = X.pop("aqi_next_day")

In [164]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor()
gb.fit(X, y)

GradientBoostingRegressor()

In [166]:
X.iloc[0]

city                          0
aqi                          19
date              1663632000000
iaqi_h                 0.178844
iaqi_p                 0.413309
iaqi_pm10             -0.083022
iaqi_t                 -0.39712
o3_avg                -0.376496
o3_max                -0.651467
o3_min                -0.192239
pm10_avg              -0.176988
pm10_max               -0.35667
pm10_min              -0.594113
pm25_avg               -0.46819
pm25_max              -0.523035
pm25_min              -0.497913
uvi_avg               -0.282843
uvi_max               -0.282843
uvi_min               -0.282843
tempmax                0.155714
tempmin               -0.069925
temp                   0.048407
feelslikemax           0.155714
feelslikemin          -0.190487
feelslike              0.056074
dew                   -0.193809
humidity              -0.356076
precip                -0.540161
precipprob            -1.071247
precipcover           -0.771696
snow                          0
snowdept

In [173]:
gb.predict(X.iloc[0].values.reshape(1, -1))

array([7.22552946])

In [174]:
y.iloc[0]

7

In [175]:
gb.predict(X.iloc[5].values.reshape(1, -1))

array([3.15480916])

In [176]:
y.iloc[5]

3

## <span style='color:#ff5f27'>👮🏼‍♀️ Model Registry</span>

In [177]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [178]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'city', 'type': 'object'},
   {'name': 'aqi', 'type': 'object'},
   {'name': 'date', 'type': 'object'},
   {'name': 'iaqi_h', 'type': 'float64'},
   {'name': 'iaqi_p', 'type': 'float64'},
   {'name': 'iaqi_pm10', 'type': 'float64'},
   {'name': 'iaqi_t', 'type': 'float64'},
   {'name': 'o3_avg', 'type': 'float64'},
   {'name': 'o3_max', 'type': 'float64'},
   {'name': 'o3_min', 'type': 'float64'},
   {'name': 'pm10_avg', 'type': 'float64'},
   {'name': 'pm10_max', 'type': 'float64'},
   {'name': 'pm10_min', 'type': 'float64'},
   {'name': 'pm25_avg', 'type': 'float64'},
   {'name': 'pm25_max', 'type': 'float64'},
   {'name': 'pm25_min', 'type': 'float64'},
   {'name': 'uvi_avg', 'type': 'float64'},
   {'name': 'uvi_max', 'type': 'float64'},
   {'name': 'uvi_min', 'type': 'float64'},
   {'name': 'tempmax', 'type': 'float64'},
   {'name': 'tempmin', 'type': 'float64'},
   {'name': 'temp', 'type': 'float64'},
   {'name': 'feelslikemax', 'type

In [180]:
import joblib

joblib.dump(gb, 'model.pkl')

['model.pkl']

In [181]:
model = mr.sklearn.create_model(
    name="gradient_boost_model",
    # we have very few of observations
    metrics={"f1": "0.5"},
    description="Gradient Boost Regressor.",
    input_example=X.sample(),
    model_schema=model_schema
)

model.save('model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/167/models/gradient_boost_model/1


Model(name: 'gradient_boost_model', version: 1)

## <span style="color:#ff5f27;">🚀 Fetch and test the model </span>